In [110]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.test
dbcol_detail = db.ani_detail
# print(dbcol_detail.find_one({}))

In [111]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# name, genres, content, tags, series_id 불러오기
data = pd.DataFrame(list(dbcol_detail.find({},{"id":1, "name":1, "genres":1, "content":1, "tags":1, "series_id":1})))

In [112]:
# 내용을 합칠 feature라는 열 만들고 결측값들 빈 값으로 대체
data['feature'] = ''
print(data.iloc[0])
print('content 열의 결측값의 수:',data['content'].isnull().sum())
print('genres 열의 결측값의 수:',data['genres'].isnull().sum())
print('name 열의 결측값의 수:',data['name'].isnull().sum())
print('tags 열의 결측값의 수:',data['tags'].isnull().sum())
print('series_id 열의 결측값의 수:',data['series_id'].isnull().sum())
data['genres'] = data['genres'].fillna('')
data['content'] = data['content'].fillna('')
data['name'] = data['name'].fillna('')
data['tags'] = data['tags'].fillna('')
data['series_id'] = data['series_id'].fillna('')
print('content 열의 결측값의 수:',data['content'].isnull().sum())
print('genres 열의 결측값의 수:',data['genres'].isnull().sum())
print('name 열의 결측값의 수:',data['name'].isnull().sum())
print('tags 열의 결측값의 수:',data['tags'].isnull().sum())
print('series_id 열의 결측값의 수:',data['series_id'].isnull().sum())

_id                                   6322bfd5db337b16410bfe29
id                                                       39986
content      소년은 싸운다 --- '올바른 죽음'을 위해 \r\n\r\n쓰라린 고통, 후회, 치...
genres                                               [판타지, 액션]
name                                               주술회전 part 1
series_id                                               4057.0
tags         [성장, 요괴 및 괴물, 이능력, 열혈, 교복, 판타지, 액션, 학교, 마을, 작화...
feature                                                       
Name: 0, dtype: object
content 열의 결측값의 수: 14
genres 열의 결측값의 수: 14
name 열의 결측값의 수: 14
tags 열의 결측값의 수: 14
series_id 열의 결측값의 수: 2585
content 열의 결측값의 수: 0
genres 열의 결측값의 수: 0
name 열의 결측값의 수: 0
tags 열의 결측값의 수: 0
series_id 열의 결측값의 수: 0


In [113]:
# stopwords 다운로드
# import nltk
# nltk.download('stopwords')
# C:\Users\SSAFY\AppData\Roaming\nltk_data\corpora\stopwords에 korean 파일 생성 후 텍스트 추가하여 불러오기
from nltk.corpus import stopwords
stop_words = stopwords.words("korean")

In [114]:
# # 불용어 추가를 위한 빈도수 확인
# from konlpy.tag import Okt
# from collections import Counter
# from nltk.corpus import stopwords
# import re

# stop_words = stopwords.words("korean")

# def morph_and_stopword(s):
#     token_ls = []
#     #형태소 분석
#     tmp = okt.morphs(s, stem=True)
#     #불용어 처리
#     for token in tmp:
#         if token not in stop_words and len(token) > 1:
#             token_ls.append(token)
#     return token_ls


# def sub_special(s):
#     return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

# okt = Okt()
# N = len(data)
# token_result = list()
# for d in range(N):
#     temp1 = ' '.join(data.loc[d]['genres'])
#     temp2 = ' '.join(data.loc[d]['tags'])
#     data.loc[d]['feature'] = temp1 + '\n' + data.loc[d]['content']
#     data.loc[d]['feature'] = temp1 + '\n' + temp2 + '\n' + data.loc[d]['content']
#     token_result += morph_and_stopword(sub_special(data.loc[d]['feature']))
# token_result_count = Counter(token_result)
# print(token_result_count)

In [115]:
# Okt 활용
from konlpy.tag import Okt
okt = Okt()


def morph_and_stopword(s):
    token_ls = []
    #형태소 분석
    tmp = okt.morphs(s, stem=True)
    #불용어 처리
    for token in tmp:
        if token not in stop_words and len(token) > 1:
            token_ls.append(token)
    return token_ls

In [143]:
# 필터링
import re

def sub_special(s):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

# data['feature'] = data['genres'] + data['tags']
# data['feature'] = ' '.join(data['genres'] + '\n' + data['tags'])
# print(data['feature'])
N = len(data)
for d in range(N):
#     # genres와 tags는 list 형식이므로 str로 변경
    data['genres'] = ' '.join(data.loc[d]['genres']
    data['tags'] = ' '.join(data.loc[d]['tags']
#     temp1 = ' '.join(data.loc[d]['genres'].copy())
#     temp2 = ' '.join(data.loc[d]['tags'].copy())
# #     temp3 = data.loc[d]['content']
#     # genres, tags, content 합치기
#     data.loc[d]['feature'] = temp1 + '\n' + temp2 + '\n' + data.loc[d]['content']
#     data.loc[d]['feature'] = ' '.join(data.loc[d]['feature'])
#     data.loc[d]['feature'] += data.loc[d]['content']
#     # none이면 join이 불가
#     if len(data.loc[d]['feature']) > 2:
#         data.loc[d]['feature'] = ' '.join(morph_and_stopword(sub_special(data.loc[d]['feature'])))
                        


0        
1        
2        
3        
4        
       ..
6116     
6117     
6118     
6119     
6120     
Name: feature, Length: 6121, dtype: object
판타지 액션

0        
1        
2        
3        
4        
       ..
6116     
6117     
6118     
6119     
6120     
Name: feature, Length: 6121, dtype: object


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3844\131592145.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[d]['feature'] = temp1 + '\n' + temp2


In [50]:
# print(data.loc[0]['feature'])
print(data.loc[0])

_id                                   6322bfd5db337b16410bfe29
content      소년은 싸운다 --- '올바른 죽음'을 위해 \r\n\r\n쓰라린 고통, 후회, 치...
genres                                               [판타지, 액션]
name                                               주술회전 part 1
series_id                                               4057.0
tags         [성장, 요괴 및 괴물, 이능력, 열혈, 교복, 판타지, 액션, 학교, 마을, 작화...
feature      판타지 액션 성장 요괴 괴물 능력 교복 판타지 액션 학교 마을 작화 만화 블루레이 ...
Name: 0, dtype: object


In [53]:
# TF-IDF 분석
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['feature'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

# 애니 인덱스를 리턴
name_to_id = dict(zip(data['name'], data.id))
find_id = name_to_id['포켓몬스터 새벽빛의 날개']
print(find_id)
# print(cosine_sim)

TF-IDF 행렬의 크기(shape) : (6121, 25456)
코사인 유사도 연산 결과 : (6121, 6121)
6030


In [12]:
def get_recommendations(name, cosine_sim=cosine_sim):
    # id를 받아오고
    find_id = name_to_id[name]
    # 유사도를 가져오고
    sim_scores = list(enumerate(cosine_sim[find_id]))
    # 유사도에 따라 정렬하고
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 상위 10개의 튜플을 넣을 공간 ten과 같은 시리즈인지 확인하기 위한 temp_series_id 선언
    ten = list()
    temp_series_id = list()
    # 대상이 되는 애니메이션의 시리즈가 추가되지 않도록 temp_series_id에 추가
    temp_series_id.append(data['series_id'].iloc[find_id])
    for index, value in enumerate(sim_scores):
        # 유사도가 0.95를 넘지 않고 이미 시리즈가 속해져있지 않다면 추가
        if value[1] < 0.95 and (data['series_id'].iloc[sim_scores[index][0]] not in temp_series_id):
            ten.append(sim_scores[index])
            temp_series_id.append(data['series_id'].iloc[sim_scores[index][0]])
        # 10개가 채워지면 stop
        if len(ten) == 10:
            break
    # 10개의 id 뽑기
    ani_ids = [idx[0] for idx in ten]
    # 인덱스를 활용해서 제목 뽑기
    return data['name'].iloc[ani_ids]

In [48]:
get_recommendations('(더빙) 원피스 3기')
# DB에 넣는 것은 빅보카 있는 것을 보면 올려 놓은 것에서....... 빅보카에서 첫번째 쉘에 형태소 분석 함수가 있음. 그것을 참고해서.
# 리스트 값 만들고. 반복문 안에서 하나씩 집어넣었음. 그것도 행렬만들 때. ? 참고해서 보면 될듯.
# 업데이트 ? update_one -> 하나씩 수정하는거. 처음 조건 값은 아이디가 data['id']. $ 이거이거 여기에 셋하고 필드명 넣고.
# 유사도라던가 그런거 넣고. 리스트를 한 행씩 짤라서 넣으면 됨.
# 데이터 백업이 뭔데....
# 유사도에 따라 정렬해서 10개까지만 뽑아서 정렬된거에 아이디가 있으니 그것만 저장하면 될 듯
# 디테일에 넣기.

6030                 포켓몬스터 새벽빛의 날개
3819                     디지몬 어드벤처:
5176                       버츄어 파이터
1550             던전 앤 파이터 : 역전의 바퀴
3767           (자막) 요괴학원Y ~N과의 조우~
4080                     마신영웅전 와타루
2332                  (자막) 원피스 20기
3790          죠죠의 기묘한 모험 6부: 스톤 오션
875     나의 히어로 아카데미아 : 구해라! 구조 훈련!
5618                     천상천하: 과거편
Name: name, dtype: object

## 실행결과
### get_recommendations('도쿄 리벤저스 part 1')  
#### (수정전)  
544                   약캐 토모자키 군  
3        전생했더니 슬라임이었던 건에 대하여 1기  
428     전생했더니 슬라임이었던 건에 대하여 OAD  
281                        괴물사변  
4209                  텔레파시 소녀 란  
3726                    치팅 크래프트  
5363               페르소나 트리니티 소울  
6038                         버블  
4949              물드는 세계의 내일로부터  
4537                흑장미 부인의 문방구  
  
#### (stop_word 더 추가)  
544                     약캐 토모자키 군  
4537                  흑장미 부인의 문방구  
281                          괴물사변  
3          전생했더니 슬라임이었던 건에 대하여 1기  
428       전생했더니 슬라임이었던 건에 대하여 OAD  
4209                    텔레파시 소녀 란  
5363                 페르소나 트리니티 소울  
6038                           버블  
4949                물드는 세계의 내일로부터  
4390    극장판 DEEMO: 너의 연주는 마음을 수놓아  
  
#### (시리즈는 처음 나오는 하나만 나오도록 추가)  
544                      약캐 토모자키 군  
3           전생했더니 슬라임이었던 건에 대하여 1기  
5363                  페르소나 트리니티 소울  
2585                       바케모노가타리  
798                (자막) 시간을 달리는 소녀  
1537               (더빙) 시간을 달리는 소녀  
870                        목소리의 형태  
588                     진격의 거인 OAD  
875     나의 히어로 아카데미아 : 구해라! 구조 훈련!  
5445                        계 츠구모모  
  
  
### get_recommendations('(더빙) 원피스 1기')  
1877                         (자막) 원피스 스탬피드  
2262                (자막) 토리코 x 원피스 콜라보 스페셜  
2277    (자막) 토리코 x 원피스 x 드래곤볼 Z 최강 콜라보 스페셜  
3027                         (더빙) 꼬마잠수함 올리  
5111                        건강 전라계 수영부 우미쇼  
363                   Free! Eternal Summer  
4862                           코브라 디 애니메이션  
6030                         포켓몬스터 새벽빛의 날개  
2289                      극장판 뽀로로  보물섬 대모험  
4369             (자막) 스페이스 독 3 : 트로피컬 어드벤처  
  
  
### get_recommendations('(자막) 스파이 패밀리')  
2399                 도사의 무녀 : 새겨진 일섬의 등불  
5188                             애 천지무용!  
6110    이과가 사랑에 빠졌기에 증명해보았다 r=1-sinθ(하트)  
4424                  극장판 장난을 잘 치는 타카기 양  
2763                            이웃집 야마다군  
1825                           안녕 자두야 1기  
3722                                곡스 2  
3720                         비밀의 아코짱 제2기  
4634                             조이드 퓨저스  
4758                            최종시험 쿠지라  